### **InstaSwap Python Examples Notebook**

**Powered by NEAR 1Click Intents**

Libraries

In [1]:
!python --version

Python 3.12.0


In [2]:
# !pip install requests py_near py_near_primitives PyNaCl base58 loguru -q

In [3]:
import os
import time
import datetime as dt
from typing import Optional, Dict, Any
import requests
from py_near.account import Account  # use to send money
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
BASE_URL = "https://1click.chaindefuser.com"
JWT = ""
# JWT = os.getenv("ONECLICK_JWT")  # put your token in env to avoid fees

##### **1) Discover Tokens**

In [5]:
def _headers() -> Dict[str, str]:
    if not JWT:
        # You can still call the API, but without the JWT you may incur a small fee.
        # See docs for how to obtain a JWT.
        pass
    return {
        "Authorization": f"Bearer {JWT}" if JWT else "",
        "Accept": "*/*",
        "Content-Type": "application/json",
    }

def list_supported_tokens() -> Any:
    """GET /v0/tokens"""
    r = requests.get(f"{BASE_URL}/v0/tokens", headers=_headers(), timeout=30)
    r.raise_for_status()
    return r.json()

In [6]:
tokens = list_supported_tokens()

In [7]:
len(list(tokens))

94

In [8]:
print("Supported tokens (truncated):", tokens[:3])  # sample

Supported tokens (truncated): [{'assetId': 'nep141:wrap.near', 'decimals': 24, 'blockchain': 'near', 'symbol': 'wNEAR', 'price': 2.76, 'priceUpdatedAt': '2025-08-10T15:02:00.060Z', 'contractAddress': 'wrap.near'}, {'assetId': 'nep141:eth.bridge.near', 'decimals': 18, 'blockchain': 'near', 'symbol': 'ETH', 'price': 4221.07, 'priceUpdatedAt': '2025-08-10T15:02:00.060Z', 'contractAddress': 'eth.bridge.near'}, {'assetId': 'nep141:17208628f84f5d6ad33f0da3bbbeb27ffcb398eac501a31bd6ad2011e36133a1', 'decimals': 6, 'blockchain': 'near', 'symbol': 'USDC', 'price': 0.999791, 'priceUpdatedAt': '2025-08-10T15:02:00.060Z', 'contractAddress': '17208628f84f5d6ad33f0da3bbbeb27ffcb398eac501a31bd6ad2011e36133a1'}]


##### **2) DRY quote (preview pricing; no deposit address generated)**

| Parameter                             | Type   | Meaning                                                                                                                                                                             |
| ------------------------------------- | ------ | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **dry**                               | `bool` | If `True`, preview the quote without creating a deposit address. If `False`, creates a deposit address and locks route parameters.                                                  |
| **swap\_type**                        | `str`  | Trade type: `"EXACT_INPUT"` (you specify the exact input amount), `"EXACT_OUTPUT"` (you specify the exact output amount you want), or `"FLEX_INPUT"` (let the system adjust input). |
| **slippage\_bps**                     | `int`  | Slippage tolerance in **basis points** (bps). `100` = 1% max price movement allowed before failing/refunding.                                                                       |
| **origin\_asset**                     | `str`  | Asset you are sending from the origin chain, formatted as `nep141:<chain>-<address>.omft.near` or similar.                                                                          |
| **deposit\_type**                     | `str`  | How funds are deposited: `"ORIGIN_CHAIN"` (send directly on the origin chain) or `"INTENTS"` (fund via a virtual chain).                                                            |
| **destination\_asset**                | `str`  | Asset you want to receive on the destination chain (same format as `origin_asset`).                                                                                                 |
| **amount**                       | `str`  | Amount of the origin asset in its smallest unit (integer string, e.g., "1000000" for 1 USDC with 6 decimals).                                                                       |
| **refund\_to**                        | `str`  | Address that should receive refunds if the swap fails or times out.                                                                                                                 |
| **refund\_type**                      | `str`  | `"ORIGIN_CHAIN"` (refund on original chain) or `"INTENTS"` (refund via virtual chain).                                                                                              |
| **recipient**                         | `str`  | Address that will receive the destination asset. Must match `recipient_type` format.                                                                                                |
| **recipient\_type**                   | `str`  | `"DESTINATION_CHAIN"` (deliver directly on the destination chain) or `"INTENTS"` (deliver to a virtual chain).                                                                      |
| **deadline\_iso**                     | `str`  | ISO 8601 UTC timestamp for when the quote expires (e.g., `"2025-08-09T18:59:00Z"`). Required for live quotes.                                                                       |
| **referral**                          | `str`  | Optional referral code or app identifier for tracking.                                                                                                                              |
| **quote\_wait\_ms**                   | `int`  | Time to wait (ms) for route computation before returning (default `3000`).                                                                                                          |
| **app\_fees**                         | `list` | Optional per-swap fee list, each `{ "recipient": "<address>", "fee": <bps> }`.                                                                                                      |
| **virtual\_chain\_recipient**         | `str`  | (Optional) Address on a virtual chain to receive funds instead of destination chain.                                                                                                |
| **virtual\_chain\_refund\_recipient** | `str`  | (Optional) Address on a virtual chain for refunds instead of origin chain.                                                                                                          |


In [9]:
def request_quote(
    *,
    dry: bool,
    swap_type: str,             # "EXACT_INPUT" | "EXACT_OUTPUT" | "FLEX_INPUT"
    slippage_bps: int,          # e.g., 100 for 1%
    origin_asset: str,          # e.g., "nep141:arb-0x...omft.near"
    deposit_type: str,          # "ORIGIN_CHAIN" | "INTENTS"
    destination_asset: str,     # e.g., "nep141:sol-...omft.near"
    amount: str,                # smallest units (string)
    refund_to: str,             # address for refunds
    refund_type: str,           # "ORIGIN_CHAIN" | "INTENTS"
    recipient: str,             # matches recipientType
    recipient_type: str,        # "DESTINATION_CHAIN" | "INTENTS"
    deadline_iso: Optional[str] = None,  # ISO8601; required when dry=False
    referral: Optional[str] = None,
    quote_wait_ms: int = 3000,
    app_fees: Optional[list] = None,
    virtual_chain_recipient: Optional[str] = None,
    virtual_chain_refund_recipient: Optional[str] = None,
) -> Any:
    """POST /v0/quote"""
    if not deadline_iso:
        # reasonable default: now + 90 minutes
        deadline_iso = (dt.datetime.utcnow() + dt.timedelta(minutes=90)).replace(microsecond=0).isoformat() + "Z"

    payload = {
        "dry": dry,
        "swapType": swap_type,
        "slippageTolerance": slippage_bps,
        "originAsset": origin_asset,
        "depositType": deposit_type,
        "destinationAsset": destination_asset,
        "amount": amount,
        "refundTo": refund_to,
        "refundType": refund_type,
        "recipient": recipient,
        "recipientType": recipient_type,
        "deadline": deadline_iso,
        "quoteWaitingTimeMs": quote_wait_ms,
    }
    if referral:
        payload["referral"] = referral
    if app_fees:
        payload["appFees"] = app_fees
    if virtual_chain_recipient:
        payload["virtualChainRecipient"] = virtual_chain_recipient
    if virtual_chain_refund_recipient:
        payload["virtualChainRefundRecipient"] = virtual_chain_refund_recipient

    r = requests.post(f"{BASE_URL}/v0/quote", headers=_headers(), json=payload, timeout=60)
    r.raise_for_status()
    return r.json()

Test Query #1: "What is the quote for USDC on Solana for 1 USDC on Arbitrum?"

In [10]:
dry_quote = request_quote(
    dry=True,  # preview the quote without creating a deposit address
    swap_type="EXACT_INPUT",  # specify the exact input amount
    slippage_bps=100,  # 1%
    origin_asset="nep141:arb-0xaf88d065e77c8cc2239327c5edb3a432268e5831.omft.near",  # Asset (USDC) you are sending from the origin chain (Arbitrum)
    deposit_type="ORIGIN_CHAIN",  # how funds are deposited, send directly on the origin chain
    destination_asset="nep141:sol-5ce3bf3a31af18be40ba30f721101b4341690186.omft.near",  # Asset (USDC) you want to receive on the destination chain (Solana)
    amount="1000000",  # example: smallest amount unit, 1.0 token if decimals=6
    refund_to="0x2527D02599Ba641c19FEa793cD0F167589a0f10D",  # Address that should receive refunds if the swap fails or times out.
    refund_type="ORIGIN_CHAIN",
    recipient="13QkxhNMrTPxoCkRdYdJ65tFuwXPhL5gLS2Z5Nr6gjRK",  # Address that will receive the destination asset. Must match recipient_type format.
    recipient_type="DESTINATION_CHAIN",
    referral="myapp",
    quote_wait_ms=3000,
    app_fees=[{"recipient": "recipient.near", "fee": 100}],  # 100 bps = 1%
)
print("DRY quote:", dry_quote)

/tmp/ipykernel_502632/3093258275.py:24: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  deadline_iso = (dt.datetime.utcnow() + dt.timedelta(minutes=90)).replace(microsecond=0).isoformat() + "Z"


DRY quote: {'quote': {'amountIn': '1000000', 'amountInFormatted': '1.0', 'amountInUsd': '0.9998', 'minAmountIn': '1000000', 'amountOut': '589742', 'amountOutFormatted': '0.589742', 'amountOutUsd': '0.5896', 'minAmountOut': '579842', 'timeEstimate': 22}, 'quoteRequest': {'quoteWaitingTimeMs': 3000, 'dry': True, 'swapType': 'EXACT_INPUT', 'slippageTolerance': 100, 'originAsset': 'nep141:arb-0xaf88d065e77c8cc2239327c5edb3a432268e5831.omft.near', 'depositType': 'ORIGIN_CHAIN', 'destinationAsset': 'nep141:sol-5ce3bf3a31af18be40ba30f721101b4341690186.omft.near', 'amount': '1000000', 'refundTo': '0x2527D02599Ba641c19FEa793cD0F167589a0f10D', 'refundType': 'ORIGIN_CHAIN', 'recipient': '13QkxhNMrTPxoCkRdYdJ65tFuwXPhL5gLS2Z5Nr6gjRK', 'recipientType': 'DESTINATION_CHAIN', 'deadline': '2025-08-10T16:32:54.000Z', 'referral': 'myapp', 'appFees': [{'recipient': 'recipient.near', 'fee': 100}]}, 'signature': 'ed25519:4dmrTjK2sgMZMvebE3ZqKSMPQBeexQK4nSgCXZBf1k5xs5DgXw9scnECVSxavhrR1BQoMUj8xksfMHiEcok3uEA

In [11]:
dry_quote

{'quote': {'amountIn': '1000000',
  'amountInFormatted': '1.0',
  'amountInUsd': '0.9998',
  'minAmountIn': '1000000',
  'amountOut': '589742',
  'amountOutFormatted': '0.589742',
  'amountOutUsd': '0.5896',
  'minAmountOut': '579842',
  'timeEstimate': 22},
 'quoteRequest': {'quoteWaitingTimeMs': 3000,
  'dry': True,
  'swapType': 'EXACT_INPUT',
  'slippageTolerance': 100,
  'originAsset': 'nep141:arb-0xaf88d065e77c8cc2239327c5edb3a432268e5831.omft.near',
  'depositType': 'ORIGIN_CHAIN',
  'destinationAsset': 'nep141:sol-5ce3bf3a31af18be40ba30f721101b4341690186.omft.near',
  'amount': '1000000',
  'refundTo': '0x2527D02599Ba641c19FEa793cD0F167589a0f10D',
  'refundType': 'ORIGIN_CHAIN',
  'recipient': '13QkxhNMrTPxoCkRdYdJ65tFuwXPhL5gLS2Z5Nr6gjRK',
  'recipientType': 'DESTINATION_CHAIN',
  'deadline': '2025-08-10T16:32:54.000Z',
  'referral': 'myapp',
  'appFees': [{'recipient': 'recipient.near', 'fee': 100}]},
 'signature': 'ed25519:4dmrTjK2sgMZMvebE3ZqKSMPQBeexQK4nSgCXZBf1k5xs5DgXw9s

Test Query #2: "What is the quote for SOL on Solana for 5 USDC on NEAR?"

In [12]:
# dry_quote = request_quote(
#     dry=True,  # preview the quote without creating a deposit address
#     swap_type="EXACT_INPUT",  # specify the exact input amount
#     slippage_bps=100,  # 1%
#     origin_asset="nep141:17208628f84f5d6ad33f0da3bbbeb27ffcb398eac501a31bd6ad2011e36133a1",  # Asset (USDC) you are sending from the origin chain (NEAR)
#     deposit_type="ORIGIN_CHAIN",  # how funds are deposited, send directly on the origin chain
#     destination_asset="nep141:sol.omft.near",  # Asset (SOL) you want to receive on the destination chain (Solana)
#     amount=str(5*10**24),  # example: 100000 token if decimals=6 like in Ethereum wei, or 10^24 for NEAR
#     # amount="5",  # example: 100000 token if decimals=6 like in Ethereum wei, or 10^24 for NEAR
#     refund_to="temporai.near",  # Address that should receive refunds if the swap fails or times out.
#     refund_type="ORIGIN_CHAIN",
#     recipient="eGi2M1awdgWNTB4EJC87DesHUyDgHxHS8e5bEcTyc3E",  # Address that will receive the destination asset. Must match recipient_type format.
#     recipient_type="DESTINATION_CHAIN",
#     referral="myapp",
#     quote_wait_ms=3000,
#     app_fees=[{"recipient": "recipient.near", "fee": 100}],  # 100 bps = 1%
# )
# print("DRY quote:", dry_quote)

##### **3) Live quote (generates a unique deposit address you will fund)**

Setup NEAR account info

In [13]:
account_id = os.getenv("NEAR_ACCOUNT_ID")
private_key = os.getenv("NEAR_PRIVATE_KEY")
acc = Account(account_id, private_key)

Test Quote #3: "What is the quote for 0.01 NEAR on NEAR for USDC on Arbitrium?"

In [14]:
amount = "1" + "0"*22
amount

'10000000000000000000000'

In [15]:
live_quote = request_quote(
    dry=False,
    swap_type="EXACT_INPUT",
    slippage_bps=100,
    origin_asset="nep141:wrap.near",
    deposit_type="ORIGIN_CHAIN",
    destination_asset="nep141:arb-0xaf88d065e77c8cc2239327c5edb3a432268e5831.omft.near",
    amount=amount,  # 0.01 NEAR, 24 decimals for NEAR
    refund_to=account_id,
    refund_type="ORIGIN_CHAIN",
    recipient="0xf00A3bA7C7f1840D50f243Cc0527D1425aE497C2",
    recipient_type="DESTINATION_CHAIN",
    referral="myapp",
)
print("LIVE quote:", live_quote)

/tmp/ipykernel_502632/3093258275.py:24: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  deadline_iso = (dt.datetime.utcnow() + dt.timedelta(minutes=90)).replace(microsecond=0).isoformat() + "Z"


LIVE quote: {'quote': {'amountIn': '10000000000000000000000', 'amountInFormatted': '0.01', 'amountInUsd': '0.0276', 'minAmountIn': '10000000000000000000000', 'amountOut': '22251', 'amountOutFormatted': '0.022251', 'amountOutUsd': '0.0222', 'minAmountOut': '21975', 'timeEstimate': 19, 'deadline': '2025-08-11T15:03:02.852Z', 'timeWhenInactive': '2025-08-11T15:03:02.852Z', 'depositAddress': '408ca48b5133a7609c402c1d74084218bb21d25df1dc11c83c0e32e11f97922e'}, 'quoteRequest': {'quoteWaitingTimeMs': 3000, 'dry': False, 'swapType': 'EXACT_INPUT', 'slippageTolerance': 100, 'originAsset': 'nep141:wrap.near', 'depositType': 'ORIGIN_CHAIN', 'destinationAsset': 'nep141:arb-0xaf88d065e77c8cc2239327c5edb3a432268e5831.omft.near', 'amount': '10000000000000000000000', 'refundTo': 'temporai.near', 'refundType': 'ORIGIN_CHAIN', 'recipient': '0xf00A3bA7C7f1840D50f243Cc0527D1425aE497C2', 'recipientType': 'DESTINATION_CHAIN', 'deadline': '2025-08-10T16:32:58.000Z', 'referral': 'myapp'}, 'signature': 'ed2551

Get deposit address

In [16]:
deposit_address = live_quote["quote"]["depositAddress"]
print("Deposit to:", deposit_address)

Deposit to: 408ca48b5133a7609c402c1d74084218bb21d25df1dc11c83c0e32e11f97922e


##### **4) Transfer NEAR to Deposit Address**

In [ ]:
tx = await acc.send_money(deposit_address, int(amount))  # send 0.01 NEAR to deposit address

In [ ]:
tx.transaction.hash, tx.status

('FYyqZaUU1M24dPBCvxBkzvrevFRjpZuxcCxWSHeczocq', {'SuccessValue': ''})

##### **5) Get status of the deposited NEAR**

In [ ]:
def submit_deposit_tx(tx_hash: str, deposit_address: str) -> Any:
    """POST /v0/deposit/submit  (optional but speeds up processing)"""
    payload = {"txHash": tx_hash, "depositAddress": deposit_address}
    r = requests.post(f"{BASE_URL}/v0/deposit/submit", headers=_headers(), json=payload, timeout=30)
    r.raise_for_status()
    return r.json()

def get_status(deposit_address: str) -> Any:
    """GET /v0/status?depositAddress=..."""
    r = requests.get(f"{BASE_URL}/v0/status", headers=_headers(), params={"depositAddress": deposit_address}, timeout=30)
    r.raise_for_status()
    return r.json()

In [ ]:
get_status(deposit_address)

{'status': 'SUCCESS',
 'updatedAt': '2025-08-10T14:57:38.000Z',
 'swapDetails': {'intentHashes': ['Ge7kjcKrfj6igreH1MsNgDXJZcXS7FgKty2MSth8YuZ4'],
  'nearTxHashes': ['7HRcxRwxRFLDErNAmrxamciz7ko6Tm5wu8kdspjPCToD',
   'CVh8EhiJDrLxZVxT3DszzRakEAFfMvHytcn6j3dPmu7H'],
  'amountIn': '10000000000000000000000',
  'amountInFormatted': '0.01',
  'amountInUsd': '0.0276',
  'amountOut': '22285',
  'amountOutFormatted': '0.022285',
  'amountOutUsd': '0.0223',
  'slippage': 7,
  'refundedAmount': '0',
  'refundedAmountFormatted': '0',
  'refundedAmountUsd': '0',
  'originChainTxHashes': [],
  'destinationChainTxHashes': [{'hash': '0xd891ece8f930c7d5774e5b3fa159dbe2c09f26c4a39193bf274e44589fce65d9',
    'explorerUrl': ''}]},
 'quoteResponse': {'timestamp': '2025-08-10T14:53:55.427Z',
  'signature': 'ed25519:63hRLzeMXRAJe61tufxhw9zQxP5DCfPNGPQu6Kd42fCP1wUZWbtUmYD3GiNuUnpfwcnmQ9bzSED7dGfu699AJjG9',
  'quoteRequest': {'dry': False,
   'swapType': 'EXACT_INPUT',
   'slippageTolerance': 100,
   'originA

In [ ]:
# submit_res = submit_deposit_tx(tx_hash="0x123abc456def789", deposit_address=deposit_address)

##### **Example Usage**

In [ ]:

# # -------------------------
# # Example usage (adjust as needed)
# # -------------------------
# if __name__ == "__main__":
#     # 1) Discover tokens
#     tokens = list_supported_tokens()
#     print("Supported tokens (truncated):", tokens[:3])  # sample

#     # 2) DRY quote (preview pricing; no deposit address generated)
#     dry_quote = request_quote(
#         dry=True,
#         swap_type="EXACT_INPUT",
#         slippage_bps=100,  # 1%
#         origin_asset="nep141:arb-0xaf88d065e77c8cc2239327c5edb3a432268e5831.omft.near",
#         deposit_type="ORIGIN_CHAIN",
#         destination_asset="nep141:sol-5ce3bf3a31af18be40ba30f721101b4341690186.omft.near",
#         amount_wei="1000000",  # example: 1.0 token if decimals=6
#         refund_to="0x2527D02599Ba641c19FEa793cD0F167589a0f10D",
#         refund_type="ORIGIN_CHAIN",
#         recipient="13QkxhNMrTPxoCkRdYdJ65tFuwXPhL5gLS2Z5Nr6gjRK",
#         recipient_type="DESTINATION_CHAIN",
#         referral="myapp",
#         quote_wait_ms=3000,
#         app_fees=[{"recipient": "recipient.near", "fee": 100}],  # 100 bps = 1%
#     )
#     print("DRY quote:", dry_quote)

#     # 3) Live quote (generates a unique deposit address you will fund)
#     live_quote = request_quote(
#         dry=False,
#         swap_type="EXACT_INPUT",
#         slippage_bps=100,
#         origin_asset="nep141:arb-0xaf88d065e77c8cc2239327c5edb3a432268e5831.omft.near",
#         deposit_type="ORIGIN_CHAIN",
#         destination_asset="nep141:sol-5ce3bf3a31af18be40ba30f721101b4341690186.omft.near",
#         amount_wei="1000000",
#         refund_to="0x2527D02599Ba641c19FEa793cD0F167589a0f10D",
#         refund_type="ORIGIN_CHAIN",
#         recipient="13QkxhNMrTPxoCkRdYdJ65tFuwXPhL5gLS2Z5Nr6gjRK",
#         recipient_type="DESTINATION_CHAIN",
#         referral="myapp",
#     )
#     deposit_address = live_quote["quote"]["depositAddress"]
#     print("Deposit to:", deposit_address)

#     # 4) (You send tokens to deposit_address on the origin chain)
#     # Optional: tell 1Click the tx hash immediately to speed up processing:
#     # submit_res = submit_deposit_tx(tx_hash="0x123abc456def789", deposit_address=deposit_address)
#     # print("Submitted deposit tx:", submit_res)

#     # 5) Poll for status until success/refund
#     for _ in range(30):
#         status = get_status(deposit_address)
#         print("Status:", status.get("status"))
#         if status.get("status") in {"SUCCESS", "REFUNDED", "FAILED"}:
#             break
#         time.sleep(10)